In [2]:
import json
import numpy as np
import pandas as pd

In [15]:
with open("../config.json", "r") as f:
    config = json.load(f)

### Meta

In [17]:
# change config["meta"]'s model_type and model_name'
config["meta"]["model_type"] = "lgbm"
config["meta"]["model_name"] = "lgbm-1"

In [18]:
features = config["data_columns"]["gen3_features"]

In [19]:
config["data"]["transformed"].keys()

dict_keys(['xf_train_df', 'xf_valid_df', 'xf_test_df', 'xf_seq_train_df', 'xf_seq_valid_df', 'xf_seq_test_df'])

### Class imbalance

In [22]:
train_df = pd.read_parquet(config["data"]["transformed"]["xf_train_df"])

In [23]:
target_col = config["data_columns"]["target_cols"][0]
neg, pos = np.bincount(train_df[target_col])
total = neg + pos
print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))

weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}
class_weight

Examples:
    Total: 116290
    Positive: 8079 (6.95% of total)



{0: 0.5373298463187661, 1: 7.197054090852829}

In [24]:
# meta
config["meta"]["model_type"] = "lgbm"
config["meta"]["model_name"] = f"lgbm-1"

# change config["model_params"]
params = {}
params["n_features"] = len(features)
params["n_classes"] = 2
params["nhids"] = nhid
params["init_bias_by_target"] = ib
params["metrics"] = ["AUC"]
params["loss"] = "CategoricalCrossentropy"
params["optim"] = "adam"
params["patience"] = 5
params["epochs"] = 20
params["lr"] = 3e-3
params["batch_size"] = 256
params["class_weight"] = cw
decay_per_n_epoch = 1  # decay once per two epochs
params["lr_decay_steps"] = (100000 // params["batch_size"]) * decay_per_n_epoch
params["lr_decay_rate"] = 0.98

config["model_params"] = params

with open(f"lgbm-config-1.json", "w") as f:
    json.dump(config, f, indent=4)
    
with open(f"lgbm-template.json", "w") as f:
    json.dump(config, f, indent=4)


In [14]:
# i = 1
# nhids = [[200, 200, 200],
#          [200, 200],
#          [200, 200, 200, 200],
#          [100, 100, 100]]
# class_weights = [class_weight, None]
# init_bias_by_target = [True, False]

# for nhid in nhids:
#     for cw in class_weights:
#         for ib in init_bias_by_target:
#             # meta
#             config["meta"]["model_type"] = "mlp"
#             config["meta"]["model_name"] = f"mlp-{i}"

#             # change config["model_params"]
#             params = {}
#             params["n_features"] = len(features)
#             params["n_classes"] = 2
#             params["nhids"] = nhid
#             params["init_bias_by_target"] = ib
#             params["metrics"] = ["AUC"]
#             params["loss"] = "CategoricalCrossentropy"
#             params["optim"] = "adam"
#             params["patience"] = 5
#             params["epochs"] = 20
#             params["lr"] = 3e-3
#             params["batch_size"] = 256
#             params["class_weight"] = cw
#             decay_per_n_epoch = 1  # decay once per two epochs
#             params["lr_decay_steps"] = (100000 // params["batch_size"]) * decay_per_n_epoch
#             params["lr_decay_rate"] = 0.98

#             config["model_params"] = params

#             with open(f"mlp-config-{i}.json", "w") as f:
#                 json.dump(config, f, indent=4)
            
#             i += 1

In [ ]:
config["model_params"]